# 🌟 Advanced Environmental Data Visualization
## Interactive Multi-dimensional Analysis Dashboard

This notebook demonstrates advanced data visualization techniques using Plotly, featuring:
- 🎯 Interactive 3D visualizations
- 📊 Dynamic filtering and drill-downs
- 🔍 Real-time data exploration
- 🎨 Professional styling and theming

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Set modern color theme
colors = {
    'background': '#1a1a1a',
    'text': '#ffffff',
    'grid': '#333333',
    'primary': '#00ffff',
    'secondary': '#ff69b4',
    'accent': '#32cd32'
}

## 📊 Data Preparation
Generate multi-dimensional environmental data with temporal and spatial components

In [2]:
# Generate enhanced sample data
np.random.seed(42)
n_samples = 1000

# Create sample dataset with more realistic ranges
data = {
    'Temperature': np.random.normal(25, 5, n_samples),  # °C
    'Humidity': np.clip(np.random.normal(60, 15, n_samples), 0, 100),  # %
    'Pressure': np.random.normal(1013, 10, n_samples),  # hPa
    'Wind_Speed': np.abs(np.random.normal(15, 5, n_samples)),  # m/s
    'Air_Quality': np.clip(np.random.normal(50, 20, n_samples), 0, 100),  # AQI
    'Solar_Radiation': np.abs(np.random.normal(500, 200, n_samples)),  # W/m²
    'Time': pd.date_range(start='2025-01-01', periods=n_samples, freq='h'),
    'Location': np.random.choice(['North', 'South', 'East', 'West'], n_samples),
    'Season': np.random.choice(['Spring', 'Summer', 'Fall', 'Winter'], n_samples)
}

df = pd.DataFrame(data)

# Add calculated columns
df['Feels_Like'] = df['Temperature'] + 0.33 * (df['Humidity']/100 * 6.105 * np.exp(17.27 * df['Temperature']/(237.7 + df['Temperature'])))
df['Wind_Category'] = pd.cut(df['Wind_Speed'], 
                            bins=[0, 5, 10, 15, 20, np.inf], 
                            labels=['Calm', 'Light', 'Moderate', 'Strong', 'Storm'])

df.head()

,Temperature,Humidity,Pressure,Wind_Speed,Air_Quality,Solar_Radiation,Time,Location,Season,Feels_Like,Wind_Category
0,27.483571,80.990332,1006.248217,5.460962,32.730128,415.248064,2025-01-01 00:00:00,South,Summer,37.255016,Light
1,24.308678,73.869505,1011.554813,10.698075,49.375930,409.317178,2025-01-01 01:00:00,North,Summer,31.696655,Moderate
2,28.238443,60.894456,1005.075801,12.931972,50.360337,140.871365,2025-01-01 02:00:00,East,Spring,35.915355,Moderate
3,32.615149,50.295948,1009.920385,24.438438,59.452607,433.981962,2025-01-01 03:00:00,South,Summer,40.756281,Storm
4,23.829233,70.473350,994.063853,17.782766,22.662833,646.565816,2025-01-01 04:00:00,East,Summer,30.677981,Strong


## 🎨 Interactive Multi-dimensional Dashboard

In [3]:
# Create main dashboard
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scene'}, {'type': 'heatmap'}],
           [{'colspan': 2}, None]],
    subplot_titles=(
        '3D Environmental Parameters',
        'Parameter Correlations',
        'Time Series Analysis'
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# 1. Enhanced 3D Scatter Plot
scatter3d = go.Scatter3d(
    x=df['Temperature'],
    y=df['Humidity'],
    z=df['Pressure'],
    mode='markers',
    marker=dict(
        size=df['Wind_Speed']/2,
        color=df['Air_Quality'],
        colorscale='Viridis',
        opacity=0.8,
        showscale=True,
        colorbar=dict(title='Air Quality Index')
    ),
    hovertemplate=(
        '<b>Temperature</b>: %{x:.1f}°C<br>' +
        '<b>Humidity</b>: %{y:.1f}%<br>' +
        '<b>Pressure</b>: %{z:.1f}hPa<br>' +
        '<b>Wind Speed</b>: %{marker.size:.1f}m/s<br>' +
        '<b>Air Quality</b>: %{marker.color:.1f}<br>' +
        '<extra></extra>'
    )
)
fig.add_trace(scatter3d, row=1, col=1)

# 2. Enhanced Correlation Heatmap
numeric_cols = ['Temperature', 'Humidity', 'Pressure', 'Wind_Speed', 'Air_Quality', 'Solar_Radiation', 'Feels_Like']
corr_matrix = df[numeric_cols].corr()
heatmap = go.Heatmap(
    z=corr_matrix,
    x=numeric_cols,
    y=numeric_cols,
    colorscale='RdBu',
    zmid=0,
    text=np.round(corr_matrix, 2),
    texttemplate='%{text}',
    textfont={"size": 10},
    hoverongaps=False,
    hovertemplate='%{x} vs %{y}<br>Correlation: %{z:.2f}<extra></extra>'
)
fig.add_trace(heatmap, row=1, col=2)

# 3. Enhanced Time Series with Multiple Parameters
fig.add_trace(
    go.Scatter(
        x=df['Time'],
        y=df['Temperature'],
        name='Temperature',
        line=dict(color=colors['primary'], width=2),
        hovertemplate='<b>Time</b>: %{x}<br><b>Temperature</b>: %{y:.1f}°C<extra></extra>'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['Time'],
        y=df['Feels_Like'],
        name='Feels Like',
        line=dict(color=colors['secondary'], width=2, dash='dot'),
        hovertemplate='<b>Time</b>: %{x}<br><b>Feels Like</b>: %{y:.1f}°C<extra></extra>'
    ),
    row=2, col=1
)

# Update layout with enhanced styling
fig.update_layout(
    template='plotly_dark',
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background'],
    title={
        'text': '🌍 Environmental Monitoring Dashboard',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24, 'color': colors['text']}
    },
    height=1000,
    width=1400,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor='rgba(0,0,0,0.5)'
    )
)

# Update 3D scatter layout
fig.update_scenes(
    bgcolor=colors['background'],
    xaxis=dict(gridcolor=colors['grid'], title='Temperature (°C)'),
    yaxis=dict(gridcolor=colors['grid'], title='Humidity (%)'),
    zaxis=dict(gridcolor=colors['grid'], title='Pressure (hPa)')
)

# Update 2D axes
fig.update_xaxes(gridcolor=colors['grid'], showgrid=True)
fig.update_yaxes(gridcolor=colors['grid'], showgrid=True)

fig.show()

## 🔍 Interactive Data Exploration

In [4]:
# Create interactive parameter analysis
def create_parameter_analysis(parameter, color_by='Season'):
    fig = px.box(df, 
                 x='Location', 
                 y=parameter,
                 color=color_by,
                 points='all',
                 title=f'{parameter} Distribution by Location and {color_by}',
                 template='plotly_dark')
    
    fig.update_layout(
        paper_bgcolor=colors['background'],
        plot_bgcolor=colors['background'],
        height=500
    )
    
    return fig

# Example usage
create_parameter_analysis('Temperature').show()

## 🌪️ Wind Analysis Dashboard

In [5]:
# Create wind rose chart
wind_data = df.groupby(['Location', 'Wind_Category']).size().reset_index(name='count')

fig = px.bar_polar(wind_data, 
                   r='count',
                   theta='Location',
                   color='Wind_Category',
                   title='Wind Speed Distribution by Location',
                   template='plotly_dark',
                   color_discrete_sequence=px.colors.sequential.Plasma_r)

fig.update_layout(
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background'],
    polar=dict(
        radialaxis=dict(gridcolor=colors['grid']),
        angularaxis=dict(gridcolor=colors['grid'])
    )
)

fig.show()

C:\Users\abdo\AppData\Local\Temp\ipykernel_2424\1904937448.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

locations = df['Location'].unique()
fig = go.Figure()

for loc in locations:
    filtered_df = df[df['Location'] == loc]
    
    fig.add_trace(go.Scatter(
        x=filtered_df["Time"], 
        y=filtered_df["Temperature"], 
        mode="lines",
        name=f"Temperature ({loc})",
        line=dict(color=colors['primary']),
        visible=(loc == "North")
    ))
    
    fig.add_trace(go.Scatter(
        x=filtered_df["Time"], 
        y=filtered_df["Feels_Like"], 
        mode="lines",
        name=f"Feels Like ({loc})",
        line=dict(color=colors['secondary'], dash='dot'),
        visible=(loc == "North")
    ))

dropdown_buttons = []
for i, loc in enumerate(locations):
    visibility = [False] * (2 * len(locations))
    visibility[2*i] = True
    visibility[2*i + 1] = True

    dropdown_buttons.append(dict(
        label=loc,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Temperature vs Feels Like - {loc}"}]
    ))

fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        direction="down",
        x=0.01,
        xanchor="left",
        y=1.2,
        yanchor="top",
        bgcolor=colors['background'],
        font={"color": colors['text']}
    )],
    title="Temperature vs Feels Like - North",
    xaxis_title="Time",
    yaxis_title="Temperature (°C)",
    template="plotly_dark",
    height=600,
    width=1000,
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background']
)

fig.update_xaxes(gridcolor=colors['grid'])
fig.update_yaxes(gridcolor=colors['grid'])

fig.show()
